In [77]:
import datetime
import pandas as pd
import requests
import ast
from pytickersymbols import PyTickerSymbols
from pandas.tseries.offsets import BDay

pd.options.display.max_rows = 200

In [78]:
def get_stock_price(symbol, start='2010-01-01', end='2022-10-31'):
    url = "https://yahoo-finance97.p.rapidapi.com/price-customdate"

    payload = f"end={end}&symbol={symbol}&start={start}"
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "X-RapidAPI-Key": "4166cbb432msh7aca089bff7017cp12120djsn227c153fbfcc",
        "X-RapidAPI-Host": "yahoo-finance97.p.rapidapi.com"
    }
    response = requests.request("POST", url, data=payload, headers=headers)
    
    df = response.text
    df = ast.literal_eval(df)
    df = df['data']
    df = pd.DataFrame(df)
    #display(df)
    df['Date'] = df['Date'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).date())
    df.columns = [x.lower() for x in df.columns]
    df = df[[
        'date',
        'close'
    ]]
    
    return df

In [88]:
%%capture --no-display
def get_yahoo_symbol(x):
    try:
        return x[0]['yahoo']
    except:
        return None


stock_data = PyTickerSymbols()
sdax = list(stock_data.get_stocks_by_index('SDAX'))
mdax = list(stock_data.get_stocks_by_index('MDAX'))

stocks = list()
stocks.extend(sdax)
stocks.extend(mdax)

stocks = pd.DataFrame(stocks)
stocks['name'] = stocks['name'].str.lower()
stocks['name'] = stocks['name'].str.replace(' & co. kgaa', '')
stocks['name'] = stocks['name'].str.replace('&co.kgaa', '')
stocks['name'] = stocks['name'].str.replace(' & co. kg', '')
stocks['name'] = stocks['name'].str.replace('vz', '')
stocks['name'] = stocks['name'].str.replace('vna', '')
stocks.loc[stocks['name']=='strör se', 'name'] = 'ströer se'
stocks['name'] = stocks['name'].str.lower().replace(' ', '_').replace('ä', 'ae').replace('ü', 'ue').replace('ö', 'oe').replace('+', '').replace('.', '').replace('ó', 'o')

stocks['yahoo_symbol'] = stocks['symbols'].apply(lambda x: get_yahoo_symbol(x))
display(stocks.loc[0, 'symbols'])

stocks.loc[stocks['name'] == '1&1 drillisch ag', 'yahoo_symbol'] = '1U1.F'
stocks.loc[stocks['name'] == 'ceconomy ag', 'yahoo_symbol'] = 'CEC.F'
stocks.loc[stocks['name'] == 's&t ag', 'yahoo_symbol'] = None
stocks.loc[stocks['name'] == 'secunet security networks ag', 'yahoo_symbol'] = 'YSN.F'
stocks.loc[stocks['name'] == 'xing ag', 'yahoo_symbol'] = None

stocks = stocks[['name', 'yahoo_symbol', 'symbol']].drop_duplicates().dropna()
stocks = stocks.sort_values('name').reset_index(drop=True)

display(stocks)

stocks.to_csv('../data/stocks.csv', sep=';', index=False)

[{'yahoo': 'ARL.F', 'google': 'FRA:ARL', 'currency': 'EUR'},
 {'yahoo': 'AAALF', 'google': 'OTCMKTS:AAALF', 'currency': 'USD'},
 {'yahoo': 'AAALY', 'google': 'OTCMKTS:AAALY', 'currency': 'USD'},
 {'yahoo': 'ARLB.F', 'google': 'FRA:ARLB', 'currency': 'EUR'}]

,name,yahoo_symbol,symbol
0,1&1 drillisch ag,1U1.F,DRI
1,aareal bank ag,ARL.F,ARL
2,abengoa s.a.,AGOAF,AGOAF
3,ado properties sa,ADJ.F,ADJ
4,adva optical networking se,ADV.F,ADV
5,aixtron se,AIXA.F,AIXA
6,alstria office reit-ag,AOX.F,AOX
7,amadeus fire ag,AAD.F,AAD
8,aroundtown sa,AT1.F,AT1
9,atoss software ag,AOF.F,AOF


In [90]:
stock_prices = pd.DataFrame(
    index=pd.date_range(start='2010-01-01', end='2022-10-31')
)

for stock in stocks.values:
    print(stock)
    price = get_stock_price(stock[1])
    price.index = price['date']
    stock_prices[stock[0]] = price['close']

#stock_prices = stock_prices[stock_prices.index.map(BDay().is_on_offset)]

display(stock_prices)

stock_prices.to_csv('../data/stocks_prices.csv', sep=';')

['1&1 drillisch ag' '1U1.F' 'DRI']
['aareal bank ag' 'ARL.F' 'ARL']
['abengoa s.a.' 'AGOAF' 'AGOAF']
['ado properties sa' 'ADJ.F' 'ADJ']
['adva optical networking se' 'ADV.F' 'ADV']
['aixtron se' 'AIXA.F' 'AIXA']
['alstria office reit-ag' 'AOX.F' 'AOX']
['amadeus fire ag' 'AAD.F' 'AAD']
['aroundtown sa' 'AT1.F' 'AT1']
['atoss software ag' 'AOF.F' 'AOF']
['aurubis ag' 'NDA.F' 'NDA']
['auto1 group se' 'AG1.F' 'AG1']
['basler ag' 'BSL.F' 'BSL']
['baywa ag ' 'BYW6.F' 'BYW6']
['bechtle ag' 'BC8.F' 'BC8']
['bilfinger se' 'GBF.F' 'GBF']
['cancom se' 'COK.F' 'COK']
['carl zeiss meditec ag' 'AFX.F' 'AFX']
['ceconomy ag' 'CEC.F' 'CEC1']
['cewe stiftung' 'CWC.F' 'CWC']
['commerzbank ag' 'CBK.F' 'CBK']
['compugroup medical se' 'COP.F' 'COP']
['cts eventim ag' 'EVD.F' 'EVD']
['dermapharm holding se' 'DMP.F' 'DMP']
['deutsche euroshop ag' 'DEQ.F' 'DEQ']
['deutsche lufthansa ag' 'LHA.F' 'LHA']
['deutsche pfandbriefbank ag' 'PBB.F' 'PBB']
['deutz ag' 'DEZ.F' 'DEZ']
['dic asset ag' 'DIC.F' 'DIC']
['drä

/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_18828/1516087273.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_prices[stock[0]] = price['close']


['telefónica deutschland holding ag' 'O2D.F' 'O2D']


/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_18828/1516087273.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_prices[stock[0]] = price['close']


['thyssenkrupp ag' 'TKA.F' 'TKA']


...

['zeal network se' 'TIMA.F' 'TIMA']


/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_18828/1516087273.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_prices[stock[0]] = price['close']


['zooplus ag' 'ZO1.F' 'ZO1']


/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_18828/1516087273.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_prices[stock[0]] = price['close']


,1&1 drillisch ag,aareal bank ag,abengoa s.a.,ado properties sa,adva optical networking se,aixtron se,alstria office reit-ag,amadeus fire ag,aroundtown sa,atoss software ag,...,uniper se,united internet ag,vantage towers ag,varta ag,verbio vereinigte bioenergie ag,wacker chemie ag,wacker neuson se,westwing group ag,zeal network se,zooplus ag
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,4.938,11.744060,NaN,NaN,2.552000,5.340000,8.150,16.500000,NaN,6.075000,...,NaN,9.420000,NaN,14.030000,NaN,121.599998,8.33,NaN,28.299999,16.0
2010-01-05,4.840,11.897048,NaN,NaN,2.680000,5.340000,8.190,16.500000,NaN,6.250000,...,NaN,9.820000,NaN,14.400000,NaN,121.800003,8.43,NaN,28.340000,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27,12.850,32.980000,0.0001,1.587,20.459999,26.389999,7.975,104.599998,2.135,128.600006,...,3.018,18.730000,27.16,28.240000,NaN,117.699997,15.01,7.175,25.799999,NaN
2022-10-28,12.970,32.820000,0.0001,1.568,20.440001,25.510000,8.500,102.599998,2.015,126.800003,...,3.088,18.790001,26.84,27.620001,NaN,119.349998,15.16,7.050,25.650000,NaN
2022-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
